In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras
from keras import layers
from keras import metrics
from keras import optimizers
import numpy as np
import datetime

2023-05-03 01:59:59.253542: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-03 01:59:59.275822: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-03 01:59:59.652483: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/pawel/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
train, train_info = tfds.load(
    name='beans', split='train', shuffle_files=True, with_info=True)
valid, valid_info = tfds.load(
    name='beans', split='validation', shuffle_files=False, with_info=True)
test, test_info = tfds.load(
    name='beans', split='test', shuffle_files=False, with_info=True)
tfds.show_examples(test, test_info)


: 

: 

In [3]:
example_dict = next(iter(train))
print('Image shape', example_dict['image'].shape)
print('Label', example_dict['label'])


Image shape (500, 500, 3)
Label tf.Tensor(1, shape=(), dtype=int64)


2023-05-03 00:57:24.944711: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype string and shape [2]
	 [[{{node Placeholder/_1}}]]
2023-05-03 00:57:24.944905: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [2]
	 [[{{node Placeholder/_0}}]]


In [48]:
def augment(image, label):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    image = tf.image.rot90(image, tf.random.uniform(shape=[], minval=0, maxval=20, dtype=tf.int32))
    return image, label

In [81]:
def lower_resolution(images):
    return tf.image.resize(images, [224, 224])

In [7]:
def normalize_img(x):
    return (tf.cast(x['image'], tf.float32)/255.0) - 0.5, x['label']


In [86]:
ds_train = train.map(normalize_img)
ds_train = ds_train.map(augment)
ds_train = ds_train.map(lambda x, y: (lower_resolution(x), y))
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(train_info.splits['train'].num_examples)
ds_train = ds_train.batch(32)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)

In [87]:
ds_test = test.map(normalize_img)
ds_test = ds_test.map(lambda x, y: (lower_resolution(x), y))
ds_test = ds_test.cache()
ds_test = ds_test.shuffle(test_info.splits['test'].num_examples)
ds_test = ds_test.batch(32)
ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)

In [88]:
model = keras.Sequential(
    [
        tf.keras.layers.Conv2D(
            224, 3, activation=tf.keras.activations.relu, input_shape=(224, 224, 3)
        ),
        tf.keras.layers.Conv2D(224, 3, activation=tf.keras.activations.relu),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Conv2D(112, 3, activation=tf.keras.activations.relu),
        tf.keras.layers.Conv2D(112, 3, activation=tf.keras.activations.relu),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Conv2D(56, 3, activation=tf.keras.activations.relu),
        tf.keras.layers.Conv2D(56, 3, activation=tf.keras.activations.relu),
        tf.keras.layers.Conv2D(56, 3, activation=tf.keras.activations.relu),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Conv2D(28, 3, activation=tf.keras.activations.relu),
        tf.keras.layers.Conv2D(28, 3, activation=tf.keras.activations.relu),
        tf.keras.layers.Conv2D(28, 3, activation=tf.keras.activations.relu),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Conv2D(14, 3, activation=tf.keras.activations.relu),
        tf.keras.layers.Conv2D(14, 3, activation=tf.keras.activations.relu),
        tf.keras.layers.Conv2D(14, 3, activation=tf.keras.activations.relu),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(16, activation=tf.keras.activations.relu),
        tf.keras.layers.Dense(16, activation=tf.keras.activations.relu),
        tf.keras.layers.Dense(3, activation=tf.keras.activations.softmax),
    ]
)
model.summary()
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

Model: "sequential_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_140 (Conv2D)         (None, 222, 222, 224)     6272      
                                                                 
 conv2d_141 (Conv2D)         (None, 220, 220, 224)     451808    
                                                                 
 max_pooling2d_105 (MaxPooli  (None, 110, 110, 224)    0         
 ng2D)                                                           
                                                                 
 conv2d_142 (Conv2D)         (None, 108, 108, 112)     225904    
                                                                 
 conv2d_143 (Conv2D)         (None, 106, 106, 112)     113008    
                                                                 
 max_pooling2d_106 (MaxPooli  (None, 53, 53, 112)      0         
 ng2D)                                               

In [89]:
history = model.fit(ds_train, epochs=40, validation_data=ds_test)


Epoch 1/40


2023-05-03 01:45:01.766871: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_3' with dtype int64 and shape [2]
	 [[{{node Placeholder/_3}}]]
2023-05-03 01:45:01.767122: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_3' with dtype int64 and shape [2]
	 [[{{node Placeholder/_3}}]]


33/33 [==============================] - ETA: 0s - loss: 1.0583 - sparse_categorical_accuracy: 0.3859

2023-05-03 01:45:25.030670: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype string and shape [1]
	 [[{{node Placeholder/_1}}]]
2023-05-03 01:45:25.030895: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int64 and shape [1]
	 [[{{node Placeholder/_4}}]]


33/33 [==============================] - 24s 436ms/step - loss: 1.0583 - sparse_categorical_accuracy: 0.3859 - val_loss: 0.9793 - val_sparse_categorical_accuracy: 0.4922
Epoch 2/40
33/33 [==============================] - 10s 314ms/step - loss: 1.0051 - sparse_categorical_accuracy: 0.4971 - val_loss: 0.9833 - val_sparse_categorical_accuracy: 0.5547
Epoch 3/40
33/33 [==============================] - 10s 314ms/step - loss: 0.9896 - sparse_categorical_accuracy: 0.5087 - val_loss: 0.9728 - val_sparse_categorical_accuracy: 0.5703
Epoch 4/40
33/33 [==============================] - 10s 313ms/step - loss: 0.9842 - sparse_categorical_accuracy: 0.5290 - val_loss: 0.8447 - val_sparse_categorical_accuracy: 0.5938
Epoch 5/40
33/33 [==============================] - 10s 312ms/step - loss: 0.9351 - sparse_categorical_accuracy: 0.5706 - val_loss: 0.7971 - val_sparse_categorical_accuracy: 0.6172
Epoch 6/40
33/33 [==============================] - 10s 309ms/step - loss: 0.9081 - sparse_categorical_acc

KeyboardInterrupt: 

In [22]:
print(tf.config.list_physical_devices())

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
